# Fuzzy cognitive diagnosis framework (FuzzyCDF)

This notebook will show you how to train and use the FuzzyCDF.
First, we will show how to get the data (here we use Math1 from math2015 as the dataset).
Then we will show how to train a FuzzyCDF and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [FuzzyCDF.py](FuzzyCDF.ipynb)

## Data Preparation

Before we process the data, we need to first acquire the dataset which is shown in [prepare_dataset.ipynb](prepare_dataset.ipynb)

In [1]:
# Load the data from files
import numpy as np
import json

# type of problems
obj_prob_index = np.loadtxt("../../data/math2015/Math1/obj_prob_index.csv", delimiter=',', dtype=int)
sub_prob_index = np.loadtxt("../../data/math2015/Math1/sub_prob_index.csv", delimiter=',', dtype=int)
# Q matrix
q_m = np.loadtxt("../../data/math2015/Math1/q_m.csv", dtype=int, delimiter=',')
prob_num, know_num = q_m.shape[0], q_m.shape[1]

# training data
with open("../../data/math2015/Math1/train_data.json", encoding='utf-8') as file:
    train_set = json.load(file)
stu_num = max([x['user_id'] for x in train_set]) + 1
R = -1 * np.ones(shape=(stu_num, prob_num))
for log in train_set:
    R[log['user_id'], log['item_id']] = log['score']

# testing data
with open("../../data/math2015/Math1/test_data.json", encoding='utf-8') as file:
    test_set = json.load(file)

In [2]:
print(train_set[0], test_set[0])

{'user_id': 0, 'item_id': 7, 'score': 1.0} {'user_id': 0, 'item_id': 9, 'score': 1.0}


In [3]:
len(train_set), len(test_set)

(67344, 16836)

## Training and Persistence

In [4]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [6]:
from EduCDM import FuzzyCDF

cdm = FuzzyCDF(R, q_m, stu_num, prob_num, know_num, obj_prob_index, sub_prob_index, skip_value=-1)

cdm.train(epoch=10, burnin=5)
cdm.save("fuzzycdf.params")

INFO:root:save parameters to fuzzycdf.params


## Loading and Testing

In [7]:
cdm.load("fuzzycdf.params")
rmse, mae = cdm.eval(test_set)
print("RMSE: %.6f, MAE: %.6f" % (rmse, mae))

INFO:root:load parameters from fuzzycdf.params
evaluating: 100%|█████████████████████████████████████████████████████████████| 16836/16836 [00:00<00:00, 91552.55it/s]

RMSE: 0.447697, MAE: 0.405684


## Incremental Training

In [8]:
new_data = [{'user_id': 0, 'item_id': 2, 'score': 0.0}, {'user_id': 1, 'item_id': 1, 'score': 1.0}]
cdm.inc_train(new_data, epoch=10, burnin=5)